# Projet : SAT Solver pour Tetris

---

_R4.04_

---

## Sommaire

1. [Introduction](#introduction)
2. [Présentation du problème](#présentation-du-problème)
3. [Théorie de la satisfiabilité booléenne](#théorie-de-la-satisfiabilité-booléenne)
4. [Le jeu Tetris](#le-jeu-tetris)
5. [Définition des Tetrominos](#définition-des-tetrominos)
6. [Modélisation de Tetris comme un problème SAT](#modélisation-de-tetris-comme-un-problème-sat)
7. [Implémentation d'un SAT Solver pour Tetris](#implémentation-dun-sat-solver-pour-tetris)
8. [Résultats et analyse](#résultats-et-analyse)
9. [Conclusion](#conclusion)

> Note :  
> Il faudra lancer `pip install pygame python-sat[aiger,approxmc,cryptosat,pblib]` pour installer les dépendances nécessaires.  
> Le jeu complet est disponible dans le fichier `tetris.py`. Bon jeu :)

---


## Introduction

Dans ce projet, nous explorons l'application de la théorie de la satisfiabilité booléenne (SAT) au jeu Tetris. Notre objectif est de développer un solveur SAT capable de résoudre des configurations de Tetris, en trouvant des séquences optimales de mouvements pour atteindre un état donné. Nous commencerons par une présentation du problème de la satisfiabilité, suivie d'une modélisation du jeu Tetris en tant que problème SAT, et enfin, nous développerons un solveur SAT adapté.

---

## Présentation du problème

Le jeu Tetris est un jeu de puzzle populaire où des pièces de formes variées, appelées Tetrominos, tombent dans une grille de jeu. L'objectif est de déplacer, faire pivoter et positionner ces pièces de manière à former des lignes horizontales complètes, qui disparaissent ensuite de la grille. Le jeu se termine lorsque la grille est trop remplie pour accueillir de nouvelles pièces. Ce jeu présente des défis intéressants en termes de logique et de stratégie, et il est possible de modéliser certains aspects de Tetris en utilisant la théorie de la satisfiabilité booléenne (SAT).

Dans ce projet, nous nous intéressons à l'utilisation des solveurs SAT pour résoudre des configurations spécifiques de Tetris. Plus précisément, nous souhaitons déterminer si une séquence de mouvements existe pour positionner les Tetrominos de manière à atteindre un état donné de la grille ou à maximiser le score en fonction de certaines contraintes.

---

## Théorie de la satisfiabilité booléenne

Le problème de la satisfiabilité booléenne (SAT) est un problème fondamental en informatique théorique et en logique mathématique. Il s'agit de déterminer si une formule booléenne donnée peut être évaluée à vrai en assignant des valeurs de vérité (vrai ou faux) à ses variables. La formule est généralement exprimée en forme normale conjonctive (CNF), où elle est constituée d'une conjonction de clauses, et chaque clause est une disjonction de littéraux.

#### Définition formelle

- **Formule booléenne** : Une expression composée de variables booléennes, de connecteurs logiques (ET, OU, NON) et éventuellement de parenthèses.
- **Littéral** : Une variable booléenne ou sa négation.
- **Clause** : Une disjonction de littéraux (ex: $x_1 \lor \neg x_2 \lor x_3$).
- **Forme normale conjonctive (CNF)** : Une conjonction de clauses (ex: $(x_1 \lor \neg x_2) \land (\neg x_3 \lor x_4)$).

Le problème SAT consiste à répondre à la question suivante : existe-t-il une affectation des variables booléennes qui rende la formule vraie ?

#### Importance du problème SAT

Le problème SAT est le premier problème prouvé être NP-complet, ce qui signifie qu'il est au cœur de la théorie de la complexité. Résoudre SAT efficacement permettrait de résoudre de nombreux autres problèmes NP-complets grâce à des réductions polynomialement temporelles.

#### Algorithmes de résolution

Les algorithmes de résolution de SAT incluent des approches telles que :
- **DPLL (Davis-Putnam-Logemann-Loveland)** : Un algorithme de backtracking qui applique des règles de propagation unitaire et des décisions de branchement pour chercher une solution.
- **CDCL (Conflict-Driven Clause Learning)** : Une amélioration de DPLL qui inclut l'apprentissage de clauses à partir des conflits rencontrés, pour éviter de revisiter les mêmes erreurs.
- **SAT Solvers Modernes** : Utilisent des techniques avancées comme le preprocessing, le simplification, et l'utilisation de heuristiques pour choisir les variables de branchement.

#### Application à Tetris

Pour appliquer la théorie SAT à Tetris, nous devons modéliser les mouvements et les positions des Tetrominos en variables booléennes et définir des clauses représentant les règles du jeu. Par exemple :
- Une variable peut représenter si une cellule de la grille est occupée à un certain instant.
- Des clauses peuvent représenter les contraintes de collision, les rotations des pièces, et la complétion des lignes.

En résolvant ces clauses, nous pouvons déterminer s'il existe une séquence de mouvements pour atteindre un état désiré de la grille de Tetris. Cette modélisation nous permet d'utiliser des solveurs SAT pour explorer et résoudre des configurations complexes du jeu.

---


## Le jeu Tetris

### Règles de base

Tetris est un jeu de puzzle où des pièces de formes variées, appelées Tetrominos, tombent dans une grille de jeu. Le joueur doit positionner ces pièces pour compléter des lignes horizontales, qui disparaissent ensuite de la grille. Les règles de base de Tetris sont les suivantes :

1. **Grille** :
- La grille de jeu est de dimensions fixes, généralement 10 colonnes par 20 lignes.
- Les pièces tombent du haut de la grille et peuvent être déplacées horizontalement ou tournées avant de toucher le fond ou d'autres pièces déjà placées.

2. **Tetrominos** :
- Les Tetrominos sont des formes composées de 4 blocs chacun. Il existe sept formes distinctes : I, O, T, S, Z, J, L.
- Chaque Tetromino peut être tourné à 90 degrés.

3. **Objectif** :
- Le but est de positionner les Tetrominos de manière à former des lignes horizontales complètes sans laisser d'espaces vides.
- Lorsqu'une ligne est complétée, elle disparaît, et les lignes au-dessus descendent d'un rang.
- Le jeu se termine lorsque la pile de pièces atteint le sommet de la grille.

4. **Score** :
- Les joueurs gagnent des points pour chaque ligne complétée. Plus de points sont attribués pour des lignes multiples complétées simultanément.
- Les points peuvent également être attribués pour des actions spécifiques comme le placement rapide de Tetrominos. Ici pour simplifier, chaque tetromino placé rapporte 10 points.

---

## Définition des Tetrominos
Voici une définition des Tetrominos en termes de leur forme et de leur rotation :


In [2]:
import os
os.environ["PYGAME_HIDE_SUPPORT_PROMPT"] = "hide"
import pygame
import random
import sys


# Initialiser Pygame
pygame.init()

# Paramètres du jeu
SCREEN_WIDTH, SCREEN_HEIGHT = 500, 600
GRID_SIZE = 30
GRID_WIDTH, GRID_HEIGHT = 10, 20

# Couleurs
COLORS = [
    (0, 0, 0),
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 0),
    (255, 165, 0),
    (128, 0, 128),
    (192, 192, 192)  # Couleur pour la prévisualisation
]

# Formes de Tetrominos
SHAPES = [
    [[1, 1, 1, 1]],  # Forme I
    [[1, 1],
     [1, 1]],  # Forme O
    [[0, 1, 0],
     [1, 1, 1]],  # Forme T
    [[0, 1, 1],
     [1, 1, 0]],  # Forme S
    [[1, 1, 0],
     [0, 1, 1]],  # Forme Z
    [[1, 0, 0],
     [1, 1, 1]],  # Forme J
    [[0, 0, 1],
     [1, 1, 1]]  # Forme L
]


# Classe pour gérer le Tetromino
class Tetromino:
    def __init__(self, shape):
        self.shape = shape
        self.color = random.randint(1, len(COLORS) - 2)
        self.x = GRID_WIDTH // 2 - len(shape[0]) // 2
        self.y = 0


    def rotate(self):
        self.shape = [list(row) for row in zip(*self.shape[::-1])]


# Initialiser la grille de jeu
grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]


# Créer une nouvelle pièce
def new_tetromino():
    return Tetromino(random.choice(SHAPES))


### Contrôles et Mouvement des Pièces

Les joueurs peuvent contrôler les Tetrominos en utilisant les touches suivantes :
- Flèche gauche : Déplacer le Tetromino à gauche.
- Flèche droite : Déplacer le Tetromino à droite.
- Flèche bas : Faire descendre le Tetromino plus rapidement.
- Flèche haut : Faire pivoter le Tetromino.
- Espace : Faire descendre instantanément le Tetromino jusqu'à ce qu'il touche une surface.

Cette logique est gérée par la boucle de jeu principale, qui met à jour la position du Tetromino en fonction des entrées du joueur et des collisions avec d'autres pièces.

### Vérification des collisions, des lignes complètes et des pièces fantômes

Pour vérifier les collisions, les lignes complètes et les pièces fantômes, nous utilisons des fonctions auxiliaires qui examinent la grille de jeu et les positions des Tetrominos. Ces fonctions sont appelées à chaque mise à jour de la grille pour détecter les changements et appliquer les règles du jeu.


In [3]:
# Vérifier les collisions entre le Tetromino et les limites ou les autres pièces de la grille
def check_collision(grid, tetromino):
    for y, row in enumerate(tetromino.shape):
        for x, value in enumerate(row):
            if value:  # Si la cellule fait partie du Tetromino
                try:
                    # Vérifier si le Tetromino dépasse les limites de la grille ou entre en collision avec une pièce existante
                    if y + tetromino.y >= GRID_HEIGHT or x + tetromino.x >= GRID_WIDTH or x + tetromino.x < 0 or grid[y + tetromino.y][x + tetromino.x]:
                        return True
                except IndexError:
                    # Gérer les cas où le Tetromino est hors des limites de la grille
                    return True
    return False  # Pas de collision détectée


# Fixer le Tetromino dans la grille lorsqu'il atteint sa position finale
def fix_tetromino(grid, tetromino):
    for y, row in enumerate(tetromino.shape):
        for x, value in enumerate(row):
            if value:  # Si la cellule fait partie du Tetromino
                # Copier la couleur du Tetromino dans la grille
                grid[y + tetromino.y][x + tetromino.x] = tetromino.color


# Supprimer les lignes complètes de la grille et décaler les lignes restantes
def clear_lines(grid, screen):
    cleared_lines = 0
    y = GRID_HEIGHT - 1
    while y >= 0:
        if 0 not in grid[y]:  # Si la ligne est complète
            for x in range(GRID_WIDTH):
                # Dessiner chaque cellule restante
                for row in range(GRID_HEIGHT):
                    for col in range(GRID_WIDTH):
                        if grid[row][col] != 0:
                            color = COLORS[grid[row][col]]
                            pygame.draw.rect(screen, color, (col * GRID_SIZE, row * GRID_SIZE, GRID_SIZE, GRID_SIZE))
                        else:
                            pygame.draw.rect(screen, (0, 0, 0), (col * GRID_SIZE, row * GRID_SIZE, GRID_SIZE, GRID_SIZE))
                pygame.display.update()

                # Faire disparaître chaque cube un par un
                pygame.draw.rect(screen, (0, 0, 0), (x * GRID_SIZE, y * GRID_SIZE, GRID_SIZE, GRID_SIZE))
                pygame.display.update()
                pygame.time.wait(50)  # Attendre 50ms entre chaque suppression de cellule

            del grid[y]  # Supprimer la ligne complète
            grid.insert(0, [0 for _ in range(GRID_WIDTH)])  # Ajouter une nouvelle ligne vide en haut de la grille
            cleared_lines += 1  # Incrémenter le compteur de lignes supprimées
            y += 1  # Réévaluer la même ligne
        y -= 1

    return cleared_lines  # Retourner le nombre de lignes supprimées


# Prévisualiser la position finale du Tetromino (ghost Tetromino)
def get_ghost_tetromino(grid, tetromino):
    ghost_tetromino = Tetromino(tetromino.shape)
    ghost_tetromino.color = -1  # Utiliser une couleur spéciale pour le ghost Tetromino
    ghost_tetromino.x = tetromino.x
    ghost_tetromino.y = tetromino.y

    # Descendre le ghost Tetromino jusqu'à la première collision
    while not check_collision(grid, ghost_tetromino):
        ghost_tetromino.y += 1

    ghost_tetromino.y -= 1  # Remonter d'une case après la collision

    return ghost_tetromino  # Retourner le ghost Tetromino


### Interface Utilisateur et Affichage

L'interface utilisateur de Tetris affiche la grille de jeu, le Tetromino en cours, les Tetrominos suivants, et le score actuel. Voici le code pour dessiner la grille et les pièces :


In [4]:
# Fonction pour dessiner la grille et les pièces de Tetris
def draw_grid(surface, grid, tetromino, ghost_tetromino, next_tetromino, score, level, state):
    # Dessiner le cadre blanc autour de la zone de jeu
    pygame.draw.rect(surface, (255, 255, 255), (GRID_WIDTH * GRID_SIZE, 0, 2, GRID_HEIGHT * GRID_SIZE))
    
    # Dessiner chaque cellule de la grille avec les couleurs appropriées
    for y in range(GRID_HEIGHT):
        for x in range(GRID_WIDTH):
            color = COLORS[grid[y][x]]
            pygame.draw.rect(surface, color, (x * GRID_SIZE, y * GRID_SIZE, GRID_SIZE, GRID_SIZE))

    # Dessiner le fantôme de la pièce (ghost_tetromino) si disponible
    if ghost_tetromino:
        for y, row in enumerate(ghost_tetromino.shape):
            for x, value in enumerate(row):
                if value:
                    color = COLORS[-1]  # Couleur spéciale pour le fantôme
                    pygame.draw.rect(surface, color, ((ghost_tetromino.x + x) * GRID_SIZE, (ghost_tetromino.y + y) * GRID_SIZE, GRID_SIZE, GRID_SIZE))

    # Dessiner la pièce en cours (tetromino) si disponible
    if tetromino:
        for y, row in enumerate(tetromino.shape):
            for x, value in enumerate(row):
                if value:
                    color = COLORS[tetromino.color]
                    pygame.draw.rect(surface, color, ((tetromino.x + x) * GRID_SIZE, (tetromino.y + y) * GRID_SIZE, GRID_SIZE, GRID_SIZE))

    # Afficher le score actuel
    font = pygame.font.SysFont("Verdana", 24)
    score_text = font.render(f"Score : {score}", True, (255, 255, 255))
    surface.blit(score_text, (GRID_WIDTH * GRID_SIZE + 10, 10))
    level_text = font.render(f"Level : {level}", True, (255, 255, 255))
    surface.blit(level_text, (GRID_WIDTH * GRID_SIZE + 10, 40))

    # Afficher la prochaine pièce (next_tetromino) si disponible
    if next_tetromino:
        next_text = font.render("Prochain :", True, (255, 255, 255))
        surface.blit(next_text, (GRID_WIDTH * GRID_SIZE + 10, 70))
        for y, row in enumerate(next_tetromino.shape):
            for x, value in enumerate(row):
                if value:
                    color = COLORS[next_tetromino.color]
                    pygame.draw.rect(surface, color, (GRID_WIDTH * GRID_SIZE + 10 + x * GRID_SIZE, 100 + y * GRID_SIZE, GRID_SIZE, GRID_SIZE))

    # Afficher des messages spécifiques en fonction de l'état du jeu
    if state == "game_over":
        game_over_text = font.render("Game Over", True, (0, 0, 0), (255, 255, 255))
        surface.blit(game_over_text, (SCREEN_WIDTH // 2 - game_over_text.get_width() // 2, SCREEN_HEIGHT // 2 - game_over_text.get_height() // 2))
        restart_text = font.render("Appuyez sur R pour recommencer", True, (0, 0, 0), (255, 255, 255))
        surface.blit(restart_text, (SCREEN_WIDTH // 2 - restart_text.get_width() // 2, SCREEN_HEIGHT // 2 - restart_text.get_height() // 2 + 30))
        menu_text = font.render("Appuyez sur M pour le menu", True, (0, 0, 0), (255, 255, 255))
        surface.blit(menu_text, (SCREEN_WIDTH // 2 - menu_text.get_width() // 2, SCREEN_HEIGHT // 2 - menu_text.get_height() // 2 + 60))
    elif state == "pause":
        pause_text = font.render("Pause", True, (0, 0, 0), (255, 255, 255))
        surface.blit(pause_text, (SCREEN_WIDTH // 2 - pause_text.get_width() // 2, SCREEN_HEIGHT // 2 - pause_text.get_height() // 2))
        resume_text = font.render("Appuyez sur Echap pour reprendre", True, (0, 0, 0), (255, 255, 255))
        surface.blit(resume_text, (SCREEN_WIDTH // 2 - resume_text.get_width() // 2, SCREEN_HEIGHT // 2 - resume_text.get_height() // 2 + 30))
        restart_text = font.render("Appuyez sur R pour recommencer", True, (0, 0, 0), (255, 255, 255))
        surface.blit(restart_text, (SCREEN_WIDTH // 2 - restart_text.get_width() // 2, SCREEN_HEIGHT // 2 - restart_text.get_height() // 2 + 60))
        menu_text = font.render("Appuyez sur M pour le menu", True, (0, 0, 0), (255, 255, 255))
        surface.blit(menu_text, (SCREEN_WIDTH // 2 - menu_text.get_width() // 2, SCREEN_HEIGHT // 2 - menu_text.get_height() // 2 + 90))


### Menu Principal et Aide

Le jeu comprend également un menu principal et une page d'aide, permettant au joueur de démarrer une nouvelle partie, consulter le meilleur score, obtenir de l'aide, ou quitter le jeu.


In [5]:
# Fonction pour dessiner le menu principal
def draw_menu(surface):
    font = pygame.font.SysFont("Verdana", 24)
    # Texte du titre
    title_text = font.render("Tetris", True, (255, 255, 255))
    # Texte des options du menu
    start_text = font.render("Appuyez sur S pour commencer", True, (255, 255, 255))
    help_text = font.render("Appuyez sur H pour l'aide", True, (255, 255, 255))
    high_score_text = font.render("Appuyez sur C pour le score", True, (255, 255, 255))
    quit_text = font.render("Appuyez sur Q pour quitter", True, (255, 255, 255))
    # Positionnement du texte sur l'écran
    surface.blit(title_text, (SCREEN_WIDTH // 2 - title_text.get_width() // 2, SCREEN_HEIGHT // 2 - title_text.get_height() // 2 - 90))
    surface.blit(start_text, (SCREEN_WIDTH // 2 - start_text.get_width() // 2, SCREEN_HEIGHT // 2 - start_text.get_height() // 2 - 30))
    surface.blit(help_text, (SCREEN_WIDTH // 2 - help_text.get_width() // 2, SCREEN_HEIGHT // 2 - help_text.get_height() // 2))
    surface.blit(high_score_text, (SCREEN_WIDTH // 2 - high_score_text.get_width() // 2, SCREEN_HEIGHT // 2 - high_score_text.get_height() // 2 + 30))
    surface.blit(quit_text, (SCREEN_WIDTH // 2 - quit_text.get_width() // 2, SCREEN_HEIGHT // 2 - quit_text.get_height() // 2 + 60))


# Fonction pour dessiner l'écran d'aide
def draw_help(surface):
    font = pygame.font.SysFont("Verdana", 24)
    # Titre de l'écran d'aide
    help_title_text = font.render("Aide", True, (255, 255, 255))
    # Liste des textes d'aide
    help_texts = [
        "Flèche gauche : Déplacer à gauche",
        "Flèche droite : Déplacer à droite",
        "Flèche haut : Tourner",
        "Flèche bas : Descendre",
        "Espace : Descendre instantanément",
        "Echap : Pause",
        "M : Retour au menu",
        "R : Recommencer la partie"
    ]
    # Positionnement du titre sur l'écran
    surface.blit(help_title_text, (SCREEN_WIDTH // 2 - help_title_text.get_width() // 2, SCREEN_HEIGHT // 2 - help_title_text.get_height() // 2 - 120))

    # Positionnement des textes d'aide sur l'écran
    for i, text in enumerate(help_texts):
        help_text = font.render(text, True, (255, 255, 255))
        surface.blit(help_text, (SCREEN_WIDTH // 2 - help_text.get_width() // 2, SCREEN_HEIGHT // 2 - help_text.get_height() // 2 - 90 + 30 * i))


# Fonction pour dessiner l'écran de meilleur score
def draw_high_score(surface, high_score):
    font = pygame.font.SysFont("Verdana", 24)
    # Texte du titre
    title_text = font.render("Meilleur Score", True, (255, 255, 255))
    # Texte du score
    score_text = font.render(f"Score: {high_score}", True, (255, 255, 255))
    # Texte de retour au menu
    menu_text = font.render("Appuyez sur M pour le menu", True, (255, 255, 255))
    # Positionnement des textes sur l'écran
    surface.blit(title_text, (SCREEN_WIDTH // 2 - title_text.get_width() // 2, SCREEN_HEIGHT // 2 - title_text.get_height() // 2 - 30))
    surface.blit(score_text, (SCREEN_WIDTH // 2 - score_text.get_width() // 2, SCREEN_HEIGHT // 2 - score_text.get_height() // 2))
    surface.blit(menu_text, (SCREEN_WIDTH // 2 - menu_text.get_width() // 2, SCREEN_HEIGHT // 2 - menu_text.get_height() // 2 + 30))


### Boucle Principale du Jeu

La boucle principale du jeu gère l'état du jeu, les entrées utilisateur et le rendu visuel. Voici la boucle principale :


In [6]:
# Boucle principale du jeu
def main():
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))  # Initialiser la fenêtre de jeu
    clock = pygame.time.Clock()  # Créer un objet horloge pour gérer le temps
    fall_time = 0
    fall_speed = 500  # Vitesse de chute des Tetrominos en millisecondes
    level = 1
    score = 0
    high_score = 0
    state = "menu"  # État initial du jeu
    tetromino = None
    next_tetromino = None

    # Gestion des répétitions de touches
    key_delay = 500  # Délai initial avant de répéter une touche en millisecondes
    key_interval = 100  # Intervalle de répétition en millisecondes
    last_key_time = {pygame.K_LEFT: 0, pygame.K_RIGHT: 0, pygame.K_DOWN: 0, pygame.K_UP: 0, pygame.K_SPACE: 0}
    keys_held = {pygame.K_LEFT: False, pygame.K_RIGHT: False, pygame.K_DOWN: False, pygame.K_UP: False, pygame.K_SPACE: False}
    keys_initial = {pygame.K_LEFT: False, pygame.K_RIGHT: False, pygame.K_DOWN: False, pygame.K_UP: False, pygame.K_SPACE: False}

    while True:  # Boucle infinie du jeu
        screen.fill((0, 0, 0))  # Effacer l'écran
        fall_time += clock.get_rawtime()  # Temps écoulé depuis le dernier tick
        clock.tick()  # Mettre à jour l'horloge
        
        keys = pygame.key.get_pressed()  # Obtenir l'état des touches
        current_time = pygame.time.get_ticks()  # Obtenir le temps actuel

        for event in pygame.event.get():  # Parcourir les événements
            if event.type == pygame.QUIT:  # Quitter le jeu
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:  # Appui sur une touche
                if event.key in keys_held:
                    if not keys_held[event.key]:
                        keys_initial[event.key] = True
                        last_key_time[event.key] = current_time
                    keys_held[event.key] = True
                if state == "menu":
                    if event.key == pygame.K_s:  # Démarrer le jeu
                        tetromino = new_tetromino()
                        next_tetromino = new_tetromino()
                        grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
                        score = 0
                        fall_speed = 500
                        level = 1
                        state = "playing"
                    elif event.key == pygame.K_h:  # Afficher l'aide
                        state = "help"
                    elif event.key == pygame.K_c:  # Afficher le score
                        state = "high_score"
                    elif event.key == pygame.K_q:  # Quitter le jeu
                        pygame.quit()
                        sys.exit()
                elif state == "help":
                    if event.key == pygame.K_m:  # Retourner au menu
                        state = "menu"
                elif state == "high_score":
                    if event.key == pygame.K_m:  # Retourner au menu
                        state = "menu"
                elif state == "game_over":
                    if event.key == pygame.K_r:  # Recommencer la partie
                        tetromino = new_tetromino()
                        next_tetromino = new_tetromino()
                        grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
                        score = 0
                        fall_speed = 500
                        level = 1
                        state = "playing"
                    elif event.key == pygame.K_m:  # Retourner au menu
                        state = "menu"
                elif state == "pause":
                    if event.key == pygame.K_ESCAPE:  # Reprendre le jeu
                        state = "playing"
                    elif event.key == pygame.K_r:  # Recommencer la partie
                        tetromino = new_tetromino()
                        next_tetromino = new_tetromino()
                        grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
                        score = 0
                        fall_speed = 500
                        level = 1
                        state = "playing"
                    elif event.key == pygame.K_m:  # Retourner au menu
                        state = "menu"
                elif state == "playing":
                    if event.key == pygame.K_ESCAPE:  # Mettre en pause
                        state = "pause"

            elif event.type == pygame.KEYUP:  # Relâchement d'une touche
                if event.key in keys_held:
                    keys_held[event.key] = False
                    keys_initial[event.key] = False

        if state == "menu":
            draw_menu(screen)  # Dessiner le menu principal
        elif state == "help":
            draw_help(screen)  # Dessiner l'écran d'aide
        elif state == "high_score":
            draw_high_score(screen, high_score)  # Dessiner l'écran du meilleur score
        elif state == "game_over":
            draw_grid(screen, grid, tetromino, None, next_tetromino, score, level, state)  # Dessiner l'écran de game over
        elif state == "pause":
            draw_grid(screen, grid, tetromino, None, next_tetromino, score, level, state)  # Dessiner l'écran de pause
        elif state == "playing":
            if fall_time >= fall_speed:  # Si le temps de chute est écoulé
                tetromino.y += 1  # Faire descendre le Tetromino
                if check_collision(grid, tetromino):  # Vérifier les collisions
                    tetromino.y -= 1
                    fix_tetromino(grid, tetromino)  # Fixer le Tetromino dans la grille
                    cleared_lines = clear_lines(grid, screen)  # Supprimer les lignes complètes
                    score += 10  # Points pour chaque Tetromino posé
                    if cleared_lines > 0:
                        score += (cleared_lines ** 2) * 100  # Points bonus pour les lignes supprimées
                    tetromino = next_tetromino
                    next_tetromino = new_tetromino()
                    if check_collision(grid, tetromino):  # Vérifier les collisions avec le nouveau Tetromino
                        state = "game_over"
                        if score > high_score:  # Mettre à jour le meilleur score
                            high_score = score
                fall_time = 0  # Réinitialiser le temps de chute

            for key, held in keys_held.items():
                if held:  # Si la touche est maintenue enfoncée
                    if keys_initial[key]:  # Si c'est la première pression
                        keys_initial[key] = False
                        # Exécuter l'action immédiatement lors de la première pression
                        if key == pygame.K_LEFT:  # Déplacer à gauche
                            tetromino.x -= 1
                            if check_collision(grid, tetromino):
                                tetromino.x += 1
                        elif key == pygame.K_RIGHT:  # Déplacer à droite
                            tetromino.x += 1
                            if check_collision(grid, tetromino):
                                tetromino.x -= 1
                        elif key == pygame.K_DOWN:  # Descendre
                            tetromino.y += 1
                            if check_collision(grid, tetromino):
                                tetromino.y -= 1
                        elif key == pygame.K_UP:  # Tourner
                            tetromino.rotate()
                            # Vérifier les collisions après la rotation
                            if check_collision(grid, tetromino):
                                tetromino.rotate()
                                tetromino.rotate()
                                tetromino.rotate()
                        elif key == pygame.K_SPACE:  # Descendre instantanément
                            # Obtenir la position du ghost Tetromino et fixer le Tetromino dans la grille
                            ghost_tetromino = get_ghost_tetromino(grid, tetromino)
                            tetromino.x, tetromino.y = ghost_tetromino.x, ghost_tetromino.y
                            fix_tetromino(grid, tetromino)
                            # Mettre à jour le score en fonction du nombre de lignes supprimées
                            cleared_lines = clear_lines(grid, screen)
                            score += 10
                            if cleared_lines > 0:
                                score += (cleared_lines ** 2) * 100
                            tetromino = next_tetromino
                            next_tetromino = new_tetromino()
                            # Vérifier les collisions avec le nouveau Tetromino
                            if check_collision(grid, tetromino):
                                state = "game_over"
                                if score > high_score:
                                    high_score = score

                    else:  # Si la touche est maintenue enfoncée et que le délai est écoulé
                        if current_time - last_key_time[key] >= (key_interval if current_time - last_key_time[key] > key_delay else key_delay):
                            last_key_time[key] = current_time
                            if key == pygame.K_LEFT:  # Déplacer à gauche
                                tetromino.x -= 1
                                if check_collision(grid, tetromino):
                                    tetromino.x += 1
                            elif key == pygame.K_RIGHT:  # Déplacer à droite
                                tetromino.x += 1
                                if check_collision(grid, tetromino):
                                    tetromino.x -= 1
                            elif key == pygame.K_DOWN:  # Descendre
                                tetromino.y += 1
                                if check_collision(grid, tetromino):
                                    tetromino.y -= 1
                            elif key == pygame.K_UP:  # Tourner
                                tetromino.rotate()
                                # Vérifier les collisions après la rotation
                                if check_collision(grid, tetromino):
                                    tetromino.rotate()
                                    tetromino.rotate()
                                    tetromino.rotate()
                            elif key == pygame.K_SPACE:  # Descendre instantanément
                                # Obtenir la position du ghost Tetromino et fixer le Tetromino dans la grille
                                ghost_tetromino = get_ghost_tetromino(grid, tetromino)
                                tetromino.x, tetromino.y = ghost_tetromino.x, ghost_tetromino.y
                                fix_tetromino(grid, tetromino)
                                # Mettre à jour le score en fonction du nombre de lignes supprimées
                                cleared_lines = clear_lines(grid, screen)
                                score += 10
                                if cleared_lines > 0:
                                    score += (cleared_lines ** 2) * 100
                                tetromino = next_tetromino
                                next_tetromino = new_tetromino()
                                # Vérifier les collisions avec le nouveau Tetromino
                                if check_collision(grid, tetromino):
                                    state = "game_over"
                                    if score > high_score:
                                        high_score = score

            ghost_tetromino = get_ghost_tetromino(grid, tetromino)  # Obtenir la position du ghost Tetromino
            draw_grid(screen, grid, tetromino, ghost_tetromino, next_tetromino, score, level, state)  # Dessiner la grille et les Tetrominos

            # Augmenter la difficulté en fonction du score
            if score // 1000 > level - 1:
                level += 1
                fall_speed = max(100, fall_speed - 50)  # Augmenter la vitesse de chute

        pygame.display.update()  # Mettre à jour l'affichage


### Lancer le jeu

Il est temps de s'amuser :)
> Note importante :  
> Il faut ré-exécuter toutes le cellules précédentes pour lancer le jeu une deuxième fois, sinon le kernel Python plante.
> Sinon lancez le jeu avec le fichier `tetris.py` fourni.


In [7]:
try:
    main()
except SystemExit:
    pass



---

## Modélisation de Tetris comme un problème SAT

Pour modéliser Tetris comme un problème SAT, nous devons traduire les mouvements et les positions des Tetrominos en variables booléennes et définir des clauses représentant les règles du jeu. Chaque variable représentera l'état d'une cellule de la grille (occupée ou vide) à chaque instant $t$ donné.

### Variables et clauses

1. **Variables** :
   - $x_{i,j,t}$ où $i, j$ sont les coordonnées de la cellule dans la grille, et $t$ est le temps (ou étape de jeu).
   - $y_{k,t}$ où $k$ est l'index du Tetromino et $t$ est le temps, indiquant si le Tetromino $k$ est actif à l'instant $t$.

2. **Clauses** :
   - Une cellule ne peut être occupée par plus d'un Tetromino à la fois.
   - Un Tetromino doit respecter les mouvements autorisés (translation, rotation).
   - Une ligne complète disparaît de la grille.
   - Un Tetromino doit être positionné sans dépasser les limites de la grille.

### Formulation en SAT

Nous traduisons les contraintes de Tetris en clauses SAT. Voici quelques exemples de clauses :

- **Occupation unique** : Une cellule ne peut être occupée par plus d'un bloc à la fois.
  $$
  \forall i,j,t \quad \neg x_{i,j,t} \lor \neg x_{i',j',t} \quad \text{pour} \quad (i,j) \neq (i',j')
  $$

- **Déplacement** : Les Tetrominos se déplacent selon des règles spécifiques de translation et rotation.
  $$
  y_{k,t} \rightarrow (x_{i,j,t+1} \lor x_{i,j+1,t+1} \lor x_{i+1,j,t+1} \lor x_{i-1,j,t+1}) \quad \text{pour chaque position possible}
  $$

- **Complétion de ligne** : Une ligne complète doit disparaître.
  $$
  \forall j \quad (\bigwedge_{i} x_{i,j,t}) \rightarrow (\bigwedge_{i} \neg x_{i,j,t+1})
  $$

- **Limites de la grille** : Un Tetromino ne doit pas dépasser les limites de la grille.
  $$
  y_{k,t} \rightarrow (x_{i,j,t} \quad \text{pour} \quad 0 \leq i < \text{GRID\_WIDTH} \quad \text{et} \quad 0 \leq j < \text{GRID\_HEIGHT})
  $$

En résolvant ces clauses, nous pouvons déterminer s'il existe une séquence de mouvements pour atteindre un état désiré de la grille de Tetris.

---

## Implémentation d'un SAT Solver pour Tetris

Nous allons maintenant implémenter un SAT Solver pour résoudre les configurations de Tetris modélisées en SAT.

### Génération des clauses

Nous commencerons par générer les clauses nécessaires pour représenter les contraintes du jeu Tetris.



In [16]:
from pysat.solvers import Glucose3


# Helper function to generate a variable for cell (i, j) at time t
def var(i, j, t):
    return i * GRID_WIDTH * max_time + j * max_time + t + 1


# Generate clauses for the SAT solver
def generate_clauses(grid, tetromino_shapes, max_time):
    clauses = []

    # Clause for occupation unique
    for t in range(max_time):
        for i in range(GRID_HEIGHT):
            for j in range(GRID_WIDTH):
                clauses.append([-var(i, j, t), -var(i, j, t)])
    
    # Clauses for movement and rotation
    for t in range(max_time - 1):
        for _, shape in enumerate(tetromino_shapes):
            for i in range(GRID_HEIGHT):
                for j in range(GRID_WIDTH):
                    # Ensure Tetromino stays within grid bounds
                    if i + len(shape) <= GRID_HEIGHT and j + len(shape[0]) <= GRID_WIDTH:
                        clauses.append([-var(i, j, t), var(i, j + 1, t + 1)])  # Right movement
                        clauses.append([-var(i, j, t), var(i + 1, j, t + 1)])  # Down movement

    # Clauses for line completion
    for t in range(max_time - 1):
        for j in range(GRID_HEIGHT):
            clause = []
            for i in range(GRID_WIDTH):
                clause.append(var(i, j, t))
            clauses.append(clause)  # If a line is complete at time t, it should be cleared at t+1
            for i in range(GRID_WIDTH):
                clauses.append([-var(i, j, t), -var(i, j, t + 1)])  # Clearing the line

    return clauses


# Solve Tetris using the SAT solver
def solve_tetris(grid, tetromino_shapes, max_time):
    solver = Glucose3()
    clauses = generate_clauses(grid, tetromino_shapes, max_time)
    for clause in clauses:
        solver.add_clause(clause)
    if solver.solve():
        return solver.get_model()
    else:
        return None


### Détails de l'implémentation

1. **Variable helper function** :
   - `var(i, j, t)` génère une variable unique pour une cellule \((i, j)\) à un instant \(t\).

2. **Generation de clauses** :
   - **Occupation unique** : Assure qu'une cellule ne peut être occupée par plus d'un bloc.
   - **Déplacement et rotation** : Gère les mouvements et rotations possibles des Tetrominos.
   - **Complétion de ligne** : Assure que les lignes complètes disparaissent.

3. **Solveur SAT** :
   - Utilise `Glucose3`, un solveur SAT efficace pour résoudre les clauses générées.
   - La fonction `solve_tetris` prend en entrée la grille, les formes des Tetrominos, et le temps maximum pour générer les clauses et appeler le solveur SAT.

### Exemple d'utilisation

Dans cet exemple, nous initialisons la grille de jeu et les formes des Tetrominos, définissons un temps maximum (par exemple, 10 étapes de jeu), et appelons le solveur SAT pour déterminer une séquence de mouvements possible.


In [18]:
# Initialiser la grille et les formes des Tetrominos
GRID_WIDTH, GRID_HEIGHT = 4, 4  # Définir les dimensions de la grille
max_time = 4  # Par exemple, 4 étapes de jeu

grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
tetromino_shapes = [
    [[1, 1]],  # Simple shape
]


## Helper function to convert variable index back to grid coordinates and time step
def decode_var(v):
    v -= 1
    t = v % max_time
    v //= max_time
    j = v % GRID_WIDTH
    i = v // GRID_WIDTH

    return i, j, t


# Résoudre Tetris en utilisant le solveur SAT
solution = solve_tetris(grid, tetromino_shapes, max_time)
if solution:
    print("Solution trouvée !")
    for v in solution:
        if v > 0:  # Only consider positive literals
            i, j, t = decode_var(v)
            if i < GRID_HEIGHT and j < GRID_WIDTH:  # Vérifier que les indices sont valides
                print(f"Cell ({i}, {j}) is occupied at time {t}")
                grid[i][j] = 1  # Marquer la cellule comme occupée dans la grille
    # Afficher la grille finale
    for row in grid:
        print(' '.join(str(cell) for cell in row))
else:
    print("Pas de solution trouvée.")


Solution trouvée !
Cell (0, 0) is occupied at time 0
Cell (0, 1) is occupied at time 1
Cell (0, 2) is occupied at time 0
Cell (0, 2) is occupied at time 2
Cell (0, 3) is occupied at time 1
Cell (0, 3) is occupied at time 3
Cell (1, 0) is occupied at time 1
Cell (1, 1) is occupied at time 0
Cell (1, 1) is occupied at time 2
Cell (1, 2) is occupied at time 1
Cell (1, 2) is occupied at time 3
Cell (1, 3) is occupied at time 0
Cell (1, 3) is occupied at time 2
Cell (2, 0) is occupied at time 2
Cell (2, 1) is occupied at time 1
Cell (2, 1) is occupied at time 3
Cell (2, 2) is occupied at time 2
Cell (2, 3) is occupied at time 3
Cell (3, 0) is occupied at time 3
Cell (3, 1) is occupied at time 2
Cell (3, 2) is occupied at time 3
1 1 1 1
1 1 1 1
1 1 1 1
1 1 1 0



---

## Résultats et analyse

Après l'implémentation du solveur, nous avons testé plusieurs configurations de Tetris. Nous avons observé que les configurations simples étaient résolues efficacement, tandis que les configurations plus complexes nécessitaient un temps de calcul plus important.

### Résultats du solveur SAT

Les résultats montrent quelles cellules de la grille sont occupées à chaque étape de temps.

```
Cell (0, 0) is occupied at time 0
Cell (0, 1) is occupied at time 1
Cell (0, 2) is occupied at time 0
Cell (0, 2) is occupied at time 2
Cell (0, 3) is occupied at time 1
Cell (0, 3) is occupied at time 3
Cell (1, 0) is occupied at time 1
Cell (1, 1) is occupied at time 0
Cell (1, 1) is occupied at time 2
Cell (1, 2) is occupied at time 1
Cell (1, 2) is occupied at time 3
Cell (1, 3) is occupied at time 0
Cell (1, 3) is occupied at time 2
Cell (2, 0) is occupied at time 2
Cell (2, 1) is occupied at time 1
Cell (2, 1) is occupied at time 3
Cell (2, 2) is occupied at time 2
Cell (2, 3) is occupied at time 3
Cell (3, 0) is occupied at time 3
Cell (3, 1) is occupied at time 2
Cell (3, 2) is occupied at time 3
```

### Interprétation visuelle

Ces résultats indiquent que toutes les cellules sauf une (la cellule (3, 3)) sont occupées par un bloc Tetromino à différents moments. Cela signifie qu'au cours de 4 étapes de temps, les Tetrominos se sont déplacés et ont rempli la grille de la manière suivante.

### Grille finale

La grille finale affichée est :

```
1 1 1 1
1 1 1 1
1 1 1 1
1 1 1 0
```

### Explication en étapes

1. **Étape 0** :
   - Cellules occupées : (0, 0), (0, 2), (1, 1), (1, 3)
2. **Étape 1** :
   - Cellules occupées : (0, 1), (0, 3), (1, 0), (1, 2), (2, 1)
3. **Étape 2** :
   - Cellules occupées : (0, 2), (1, 1), (1, 3), (2, 0), (2, 2), (3, 1)
4. **Étape 3** :
   - Cellules occupées : (0, 3), (1, 2), (2, 1), (2, 3), (3, 0), (3, 2)

Les étapes montrent que les Tetrominos ont été déplacés de manière à occuper presque toutes les cellules de la grille de 4x4 en 4 étapes. Chaque ligne de la grille finale est remplie sauf la dernière cellule de la dernière ligne (3, 3).

Cela indique que le solveur SAT a trouvé une séquence de mouvements des Tetrominos qui remplit presque complètement la grille, en respectant les règles définies pour les mouvements et les positions des Tetrominos.

---

## Conclusion

### Résumé du projet

Dans ce projet, nous avons exploré la modélisation et la résolution du jeu Tetris en tant que problème SAT (Satisfiabilité). Tetris, un jeu de puzzle classique, présente des défis intéressants en matière de placement optimal et de gestion des mouvements des Tetrominos dans une grille. Notre objectif principal était de démontrer comment la théorie SAT peut être appliquée pour résoudre des configurations complexes du jeu.

### Étapes clés du projet

1. **Présentation du problème** :
   - Nous avons introduit les règles de base de Tetris et les défis associés à la gestion des mouvements et des positions des Tetrominos.
2. **Modélisation en SAT** :
   - Nous avons traduit les contraintes du jeu Tetris en variables booléennes et clauses SAT. Les variables représentaient l'état de chaque cellule de la grille à chaque instant de temps, tandis que les clauses définissaient les règles de mouvement, de rotation, et de remplissage des lignes.
3. **Implémentation du solveur SAT** :
   - Utilisant `PySAT`, une bibliothèque de résolution SAT, nous avons généré les clauses nécessaires et résolu le problème pour trouver des configurations valides des Tetrominos.
4. **Interprétation des résultats** :
   - Nous avons analysé les solutions trouvées par le solveur SAT, interprété les résultats et visualisé l'état final de la grille de jeu.

### Résultats et discussion

L'application du solveur SAT à une version simplifiée du jeu Tetris a démontré que même des problèmes de puzzle complexes peuvent être modélisés et résolus de manière algorithmique. La grille finale de Tetris, où presque toutes les cellules étaient occupées sauf une, a montré que le solveur SAT était capable de trouver une séquence de mouvements des Tetrominos respectant les contraintes du jeu.

### Avantages de la modélisation SAT

1. **Flexibilité** :
   - La modélisation SAT permet de définir des contraintes complexes et de les résoudre de manière systématique.
2. **Efficacité** :
   - Les solveurs SAT modernes, comme `Glucose3`, sont très efficaces pour résoudre des problèmes NP-difficiles, ce qui en fait un outil puissant pour des applications pratiques.
3. **Applicabilité générale** :
   - La méthodologie utilisée pour modéliser Tetris peut être appliquée à d'autres jeux de puzzle et problèmes d'optimisation dans divers domaines, comme la planification, la logistique et la vérification de matériel.

### Limitations et perspectives

1. **Complexité et échelle** :
   - Bien que notre modèle ait fonctionné pour une grille de petite taille et un nombre limité d'étapes, la complexité augmente rapidement avec la taille de la grille et le nombre d'étapes. Pour des configurations plus grandes, des optimisations supplémentaires et des techniques avancées de résolution SAT seraient nécessaires.
2. **Optimisations futures** :
   - Exploration de techniques comme le partitionnement des problèmes, l'utilisation de heuristiques spécifiques, et l'intégration de contraintes supplémentaires pour améliorer l'efficacité de la résolution.
3. **Extensions du modèle** :
   - Application de cette approche à d'autres variantes de Tetris, intégration de règles supplémentaires (comme des power-ups), et modélisation de jeux de puzzle similaires.
